In [ ]:
from keras import Input, layers
from keras import Model
import numpy as np

'''Модели с несколькими входами.'''

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# Входной текст - это послед-сть целых чисел переменной длины.
text_input = Input(shape=(None,), dtype = 'int32', name='text')

# Преобр-ние вх текста в послед-сть векторов с размером 64
embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)

# Преобр-ние векторов в единый вектор с помощью уровня LTSM
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(
    shape=(None,), dtype='int32', name='question'
)

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# Объединение закодированных вопроса и текста
concatenated = layers.concatenate(
    [encoded_text, encoded_question],
    axis=-1)

# Добавление классификатора softmax сверху
answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated)

# Создание экз модели с 2мя вх и одним выходом
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss= 'categorical_crossentropy',
              metrics= ['acc'])

# Создание массива Numpy с фиктивными данными
num_samples = 1000
max_length = 100

text = np.random.randint(
    1,
    text_vocabulary_size,
    size=(num_samples, max_length)
)
# К вопросам прим-ся прямое кодирование, а не преобразование в целые числа
question = np.random.randint(
    1,
    question_vocabulary_size,
    size=(num_samples, max_length)
)

answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
     x[indices[i]] = 1

# Передача списка входов
model.fit([text, question], answers, epochs=10, batch_size=128)

# Передача с помощью словаря (если были определены имена для входов)
model.fit(
    {'text': text, 'question': question},
    answers,
    epochs=10,
    batch_size=128
)

score = model.evaluate({'text': text, 'question': question}, answers)